In [1]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
# 读取数据
data = pd.read_csv("dataset\car_following_01_tracks.csv")

# 将 "Truck" 设置为 1，"Car" 设置为 0
label_encoder = LabelEncoder()
data['front_class_encoded'] = label_encoder.fit_transform(data['front_class'])
data['following_class_encoded'] = label_encoder.transform(data['following_class'])

# 选择需要进行主成分分析的特征列
features = ['distance', 'following_x', 'front_x', 'front_speed', 'following_speed',
             'front_width', 'following_width', 'front_class_encoded', 'following_class_encoded']

# 将特征列进行标准化
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[features])

# 创建 PCA 模型并拟合数据
pca = PCA(n_components=7)
principal_components = pca.fit_transform(scaled_data)

# 创建包含主成分的数据框
principal_df = pd.DataFrame(data=principal_components)


In [2]:
from hmmlearn import hmm
import pandas as pd
import numpy as np

observations = principal_df.values
num_states = 3  
model = hmm.GaussianHMM(n_components=num_states, covariance_type="full", n_iter=100)
model.fit(observations)
predicted_labels = model.predict(observations)

data['driving_style'] = predicted_labels
data.to_csv("output1.csv", index=False)